# COVID-19 Data Dashboard (Dash Example)
This notebook demonstrates how to build a COVID-19 dashboard using Dash, Plotly, and Pandas.

## Import Required Libraries
Import all necessary Python libraries for data manipulation, visualization, and building the Dash app.

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
from dash import dash_table
import plotly.express as px
import plotly.graph_objects as go

## Load and Prepare the COVID-19 Dataset
Read the COVID-19 data, clean it, and compute new daily cases for each country.

In [2]:
# Load data
df = pd.read_csv("./owid-covid-data.csv")

# Data cleanup
df['date'] = pd.to_datetime(df['date'])
df = df[~df['location'].isin([
    'World', 'Asia', 'Africa', 'Europe', 'High-income countries', 'Upper-middle-income countries',
    'Lower-middle-income countries', 'Low-income countries', 'North America', 'South America', 'European Union (27)'
])]

# Compute new cases per day for each country
df = df.sort_values(['location', 'date'])
df['new_cases'] = df.groupby('location')['total_cases'].diff().fillna(0).clip(lower=0)


## Define Variables for Dashboard Controls
Set up lists and mappings for countries and time periods to be used in dashboard controls.

In [3]:
# Variables
countries = df['location'].unique()
# Create a list of (year, month) tuples in the data
year_months = sorted(df['date'].dt.to_period('M').unique())
year_month_labels = [str(ym) for ym in year_months]
year_month_map = {i: ym for i, ym in enumerate(year_months)}
year_month_str_map = {i: str(ym) for i, ym in enumerate(year_months)}

## Define Color Mapping Function
Create a function to assign unique colors to selected countries for consistent visualization.

In [4]:
# Function that will be used to get correct colors
def get_color_map(selected_countries):
    palette = px.colors.qualitative.Plotly
    return {country: palette[i % len(palette)] for i, country in enumerate(selected_countries)}

## Build the Dash App Layout
Define the layout of the dashboard, including tabs, controls, and visualization areas.

In [5]:
app = dash.Dash(__name__, suppress_callback_exceptions=True)

app.title = "ESSA: COVID-19 Data Dashboard"
app.layout = html.Div([
    html.H2("ESSA: COVID-19 Data Dashboard", style={'textAlign': 'center', 'marginTop': '20px'}),
    dcc.Tabs(id='tabs', value='main', children=[
        dcc.Tab(label='Main', value='main', children=[
            html.Div([
                html.Div([
                    html.Div(
                        dcc.RangeSlider(
                            id='year-month-range-slider',
                            min=0,
                            max=len(year_months) - 1,
                            value=[0, len(year_months) - 1],
                            marks={i: str(ym) for i, ym in enumerate(year_months) if ym.month in [1, 6]},
                            step=1,
                            allowCross=False,
                            tooltip={"always_visible": False, "placement": "bottom"},
                        ),
                        style={'width': '100%', 'padding': '10px', 'border': '1px solid #ccc', 'borderRadius': '5px', 'marginRight': '0px'}
                    ),
                ], id='quick-summary', style={'margin': '10px 0','fontWeight': 'bold','display': 'flex','alignItems': 'center', 'width': '100%'}),
                # Display selected countries
                html.Div(id='selected-countries', style={'margin': '10px 0', 'fontWeight': 'bold'}),
                # The main content area
                html.Div([
                    # The left hand side panel with dropdown, search, and checklist
                    html.Div([
                        dcc.Dropdown(
                            id='metric-dropdown',
                            options=[
                                {'label': 'Total Deaths', 'value': 'total_deaths'},
                                {'label': 'Total Cases', 'value': 'total_cases'},
                                {'label': 'Total Vaccinations', 'value': 'total_vaccinations'},
                                {'label': 'New Cases', 'value': 'new_cases'}  # Added new option
                            ],
                            value='total_deaths',
                            clearable=False,
                            style={'marginBottom': '10px'}
                        ),
                        dcc.Input(
                            id='country-search',
                            type='text',
                            placeholder='Search country...',
                            style={'width': '100%', 'marginBottom': '10px'}
                        ),
                        dcc.Checklist(
                            id='country-checklist',
                            options=[{'label': c, 'value': c} for c in countries],
                            value=['Poland'],  # Set Poland as default
                            style={'height': '300px', 'overflowY': 'scroll', 'display': 'block'}
                        ),
                    ], style={ 'width': '10%', 'minWidth': '120px', 'maxWidth': '300px', 'marginRight': '30px', 'flexShrink': 0 }),
                    # The 2x2 grid layout for the main content
                    html.Div([
                        # 2x2 grid: left column (line plot, map), right column (leaderboard, data table)
                        html.Div([
                            # Left column
                            html.Div([
                                dcc.Graph(id='main-comparison-line-plot', style={'width': '100%', 'height': '350px'}),
                                dcc.Graph(id='country-map', style={'width': '100%', 'height': '350px', 'marginTop': '20px'})
                            ], style={'flex': 1, 'display': 'flex', 'flexDirection': 'column', 'marginRight': '20px'}),
                            # Right column
                            html.Div([
                                dcc.Graph(id='leaderboard', style={'width': '100%', 'height': '350px'}),
                                html.Div(
                                    id='data-table-container',
                                    style={'height': '350px', 'overflowY': 'auto', 'marginTop': '20px'}
                                )
                            ], style={ 'flex': 1, 'display': 'flex', 'flexDirection': 'column'})
                        ], style={ 'display': 'flex', 'flexDirection': 'row', 'width': '100%' }),
                    ], style={ 'flex': 1, 'display': 'flex', 'flexDirection': 'column', 'overflowX': 'auto' }),
                ], style={'display': 'flex', 'alignItems': 'flex-start', 'height': '100vh'}),
            ])
        ]),
        # The one country overview tab
        dcc.Tab(label='Map Only', value='map', children=[
            html.Div([
                html.Div([
                    dcc.Graph(id='country-map-only', style={'width': '100%', 'height': '350px', 'marginTop': '20px'}),
                ], style={'width': '82vw', 'marginRight': '20px'}),
                html.Div([
                    dcc.Graph(id='country-bar-metrics', style={'width': '100%', 'height': '350px', 'marginTop': '20px'})
                ], style={'width': '40vw'})
            ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'flex-start', 'height': '100vh'})
        ])
    ])
])

## Callback: Clear Search Box When Checklist Changes
Automatically clear the country search input when the checklist selection changes.

In [6]:
@app.callback(
    Output('country-search', 'value'),
    Input('country-checklist', 'value'),
    prevent_initial_call=True
)
def clear_search_on_checklist_change(_):
    return ""

## Callback: Update Country Checklist Based on Search
Filter the country checklist options according to the search input.

In [7]:
@app.callback(
    Output('country-checklist', 'options'),
    Input('country-search', 'value')
)
def update_checklist_options(search_value):
    if not search_value:
        filtered = countries
    else:
        filtered = [c for c in countries if search_value.lower() in c.lower()]
    return [{'label': c, 'value': c} for c in filtered]

## Callback: Update Main Comparison Line Plot
Update the line plot comparing selected countries and the average for the selected metric and time range.

In [8]:
@app.callback(
    Output('main-comparison-line-plot', 'figure'),
    [Input('country-checklist', 'value'),
     Input('metric-dropdown', 'value'),
     Input('year-month-range-slider', 'value')]
)
def update_line_plot(selected_countries, selected_metric, slider_range):
    start_idx, end_idx = slider_range
    start_period = year_month_map[start_idx]
    end_period = year_month_map[end_idx]
    start_date = pd.Period(start_period).start_time
    end_date = pd.Period(end_period).end_time
    dff = df[
        df['location'].isin(selected_countries) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ].copy()
    dff['year_month'] = dff['date'].dt.to_period('M').astype(str)
    # For new_cases, use sum per month; for others, use max
    if selected_metric == 'new_cases':
        metric_by_month = dff.groupby(['location', 'year_month'])[selected_metric].sum().reset_index()
    else:
        metric_by_month = dff.groupby(['location', 'year_month'])[selected_metric].max().reset_index()

    color_map = get_color_map(selected_countries)

    # Average for ALL countries (in the selected date range)
    df_all = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()
    df_all['year_month'] = df_all['date'].dt.to_period('M').astype(str)
    if selected_metric == 'new_cases':
        metric_all = df_all.groupby(['location', 'year_month'])[selected_metric].sum().reset_index()
        avg_by_month = metric_all.groupby('year_month')[selected_metric].mean().reset_index()
    else:
        metric_all = df_all.groupby(['location', 'year_month'])[selected_metric].max().reset_index()
        avg_by_month = metric_all.groupby('year_month')[selected_metric].mean().reset_index()

    fig = px.line(
        metric_by_month,
        x='year_month',
        y=selected_metric,
        color='location',
        markers=False,
        color_discrete_map=color_map,
        title='',
        labels={selected_metric: selected_metric.replace("_", " ").title(), 'year_month': 'Year-Month', 'location': 'Country'},
        template="simple_white"
    )
    # Add average line (all countries)
    fig.add_trace(
        go.Scatter(
            x=avg_by_month['year_month'],
            y=avg_by_month[selected_metric],
            mode='lines',
            name='Average',
            line=dict(color='black', width=3, dash='dash')
        )
    )
    fig.update_layout(
        showlegend=False,
        xaxis_title=None,
        yaxis_title=None
    )
    return fig

## Callback: Update Leaderboard Bar Chart
Show the top 10 countries for the selected metric and highlight selected countries.

In [9]:
@app.callback(
    Output('leaderboard', 'figure'),
    [Input('country-checklist', 'value'),
     Input('metric-dropdown', 'value'),
     Input('year-month-range-slider', 'value')]
)
def update_leaderboard(selected_countries, selected_metric, slider_range):
    start_idx, end_idx = slider_range
    start_period = year_month_map[start_idx]
    end_period = year_month_map[end_idx]
    start_date = pd.Period(start_period).start_time
    end_date = pd.Period(end_period).end_time
    dff = df[
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ].copy()
    # For new_cases, use sum; for others, use max
    if selected_metric == 'new_cases':
        agg_metric = dff.groupby('location')[selected_metric].sum().reset_index()
    else:
        agg_metric = dff.groupby('location')[selected_metric].max().reset_index()
    top10 = agg_metric.nlargest(10, selected_metric)
    selected_df = agg_metric[agg_metric['location'].isin(selected_countries)]
    combined = pd.concat([selected_df, top10]).drop_duplicates(subset=['location'])
    combined['is_selected'] = combined['location'].isin(selected_countries)
    combined = combined.sort_values(['is_selected', selected_metric], ascending=[False, False]).head(10)
    combined = combined.drop(columns='is_selected')
    palette = px.colors.qualitative.Plotly
    color_map = {}
    for i, country in enumerate(combined['location']):
        if country in selected_countries:
            color_map[country] = palette[selected_countries.index(country) % len(palette)]
        else:
            color_map[country] = 'black'
    fig = px.bar(
        combined,
        x=selected_metric,
        y='location',
        orientation='h',
        color='location',
        color_discrete_map=color_map,
        title='',
        labels={selected_metric: selected_metric.replace("_", " ").title(), 'location': 'Country'},
        template="simple_white"
    )
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, height=400, showlegend=False, yaxis_title=None)
    return fig

## Callback: Display Selected Countries
Show the currently selected countries with their assigned colors.

In [10]:
@app.callback(
    Output('selected-countries', 'children'),
    Input('country-checklist', 'value')
)
def show_selected_countries(selected):
    if not selected:
        return "No countries selected."
    color_map = get_color_map(selected)
    return [
        html.Span(
            country,
            style={'color': color_map[country], 'fontWeight': 'bold', 'marginRight': '10px'}
        )
        for country in selected
    ]

## Callback: Update Data Table
Display a data table of aggregated metrics for selected countries and time range.

In [11]:
@app.callback(
    Output('data-table-container', 'children'),
    [Input('country-checklist', 'value'),
     Input('year-month-range-slider', 'value'),
     Input('metric-dropdown', 'value')]
)
def update_table(selected_countries, slider_range, selected_metric):
    start_idx, end_idx = slider_range
    start_period = year_month_map[start_idx]
    end_period = year_month_map[end_idx]
    start_date = pd.Period(start_period).start_time
    end_date = pd.Period(end_period).end_time
    dff = df[
        (df['date'] >= start_date) &
        (df['date'] <= end_date) &
        (df['location'].isin(selected_countries))
    ].copy()
    dff['year_month'] = dff['date'].dt.to_period('M').astype(str)
    agg_columns = ['total_cases', 'total_deaths', 'total_vaccinations', 'population', 'new_cases']
    # For new_cases, sum per month; for others, sum as before
    dff_grouped = dff.groupby(['location', 'year_month'])[agg_columns].sum().reset_index()
    dff_grouped = dff_grouped[['location', 'year_month'] + agg_columns]
    if selected_metric in dff_grouped.columns:
        dff_grouped = dff_grouped.sort_values(by=selected_metric, ascending=False)
    dff_display = dff_grouped.head(1000)
    palette = px.colors.qualitative.Plotly
    color_map = {country: palette[i % len(palette)] for i, country in enumerate(selected_countries)}
    style_data_conditional = [
        {
            'if': {'filter_query': f'{{location}} = "{country}"'},
            'color': color_map[country]
        }
        for country in dff_display['location'].unique() if country in color_map
    ]
    return dash_table.DataTable(
        columns=[{"name": i.replace('_', ' ').title(), "id": i} for i in dff_display.columns],
        data=dff_display.to_dict('records'),
        style_data_conditional=style_data_conditional,
        style_table={'overflowX': 'auto', 'maxHeight': '340px', 'overflowY': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'maxWidth': '200px',
            'whiteSpace': 'normal',
            'fontWeight': 'bold'
        },
        style_header={'fontWeight': 'bold', 'backgroundColor': '#f9f9f9'},
        sort_action='native'
    )

## Callback: Update Country Map
Update the choropleth map to show the selected metric for all countries, highlighting selected ones.

In [12]:
@app.callback(
    Output('country-map', 'figure'),
    [Input('country-checklist', 'value'),
     Input('year-month-range-slider', 'value'),
     Input('metric-dropdown', 'value')]
)
def update_country_map(selected_countries, slider_range, selected_metric):
    start_idx, end_idx = slider_range
    start_period = year_month_map[start_idx]
    end_period = year_month_map[end_idx]
    start_date = pd.Period(start_period).start_time
    end_date = pd.Period(end_period).end_time
    dff = df[
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ].copy()
    dff = dff.sort_values('date').groupby('location').last().reset_index()
    if 'iso_code' not in dff.columns:
        return go.Figure()
    # For new_cases, sum over the period and merge back to keep iso_code
    if selected_metric == 'new_cases':
        dff_metric = df[
            (df['date'] >= start_date) &
            (df['date'] <= end_date)
        ].groupby('location', as_index=False)[selected_metric].sum()
        # Merge to keep iso_code and other info
        dff = pd.merge(dff[['location', 'iso_code']], dff_metric, on='location', how='left')
        dff['metric_value'] = dff[selected_metric].fillna(0)
    else:
        dff['metric_value'] = dff[selected_metric]
        dff['metric_value'] = dff['metric_value'].fillna(0)
    palette = px.colors.qualitative.Plotly
    color_map = {country: palette[i % len(palette)] for i, country in enumerate(selected_countries)}
    dff['border_color'] = dff['location'].map(lambda c: color_map[c] if c in color_map else 'rgba(0,0,0,0)')
    dff['border_width'] = dff['location'].map(lambda c: 3 if c in color_map else 0.5)
    fig = px.choropleth(
        dff,
        locations='iso_code',
        color='metric_value',
        hover_name='location',
        color_continuous_scale='Blues',
        labels={'metric_value': selected_metric.replace('_', ' ').title()},
        projection='natural earth',
        # title=f'{selected_metric.replace("_", " ").title()} by Country'
    )
    fig.update_traces(
        marker_line_color=dff['border_color'],
        marker_line_width=dff['border_width'],
        zmin=dff['metric_value'].min(),
        zmax=dff['metric_value'].max()
    )
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        coloraxis_colorbar=dict(title=selected_metric.replace('_', ' ').title()),
        showlegend=False
    )
    return fig

## Callback: Update Map-Only Tab Choropleth
Update the map in the "Map Only" tab and highlight the selected country.

In [13]:
@app.callback(
    Output('country-map-only', 'figure'),
    [Input('country-checklist', 'value'),
     Input('metric-dropdown', 'value'),
     Input('country-map-only', 'clickData')]
)
def update_country_map_only(selected_countries, selected_metric, clickData):
    dff = df.sort_values('date').groupby('location').last().reset_index()
    if 'iso_code' not in dff.columns:
        return go.Figure()
    # For new_cases, sum over all time and merge back to keep iso_code
    if selected_metric == 'new_cases':
        dff_metric = df.groupby('location', as_index=False)[selected_metric].sum()
        dff = pd.merge(dff[['location', 'iso_code']], dff_metric, on='location', how='left')
        dff['metric_value'] = dff[selected_metric].fillna(0)
    else:
        dff['metric_value'] = dff[selected_metric]
        dff['metric_value'] = dff['metric_value'].fillna(0)
    selected_iso = None
    if clickData and 'points' in clickData and clickData['points']:
        selected_iso = clickData['points'][0].get('location')
    else:
        poland_row = dff[dff['location'] == 'Poland']
        if not poland_row.empty:
            selected_iso = poland_row.iloc[0]['iso_code']
    def border_color(row):
        if selected_iso and row['iso_code'] == selected_iso:
            return 'red'
        return 'rgba(0,0,0,0.5)'
    def border_width(row):
        if selected_iso and row['iso_code'] == selected_iso:
            return 4
        return 0.5
    dff['border_color'] = dff.apply(border_color, axis=1)
    dff['border_width'] = dff.apply(border_width, axis=1)
    fig = px.choropleth(
        dff,
        locations='iso_code',
        color='metric_value',
        hover_name='location',
        color_continuous_scale='Blues',
        labels={'metric_value': selected_metric.replace('_', ' ').title()},
        projection='natural earth',
        title=f'{selected_metric.replace("_", " ").title()} by Country'
    )
    fig.update_traces(
        marker_line_color=dff['border_color'],
        marker_line_width=dff['border_width'],
        zmin=dff['metric_value'].min(),
        zmax=dff['metric_value'].max()
    )
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        coloraxis_colorbar=dict(title=selected_metric.replace('_', ' ').title()),
        showlegend=False
    )
    return fig

## Callback: Show Metrics for Selected Country on Map
Display time series of metrics for the country selected on the map.

In [14]:
@app.callback(
    Output('country-bar-metrics', 'figure'),
    [Input('country-map-only', 'clickData')]
)
def show_country_metrics_on_map_click(clickData):
    import plotly.graph_objects as go
    if not clickData or 'points' not in clickData or not clickData['points']:
        country = 'Poland'
    else:
        point = clickData['points'][0]
        iso_code = point.get('location')
        country_row = df[df['iso_code'] == iso_code]
        if country_row.empty:
            return go.Figure()
        country = country_row.iloc[0]['location']
    dff = df[df['location'] == country].copy()
    if dff.empty:
        return go.Figure()
    dff['year_month'] = dff['date'].dt.to_period('M').astype(str)
    monthly = dff.groupby('year_month').agg({
        'total_deaths': 'max',
        'total_cases': 'max',
        'total_vaccinations': 'max',
        'new_cases': 'sum'
    }).reset_index()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=monthly['year_month'], y=monthly['total_deaths'],
        mode='lines', name='Total Deaths', line=dict(color='#EF553B')
    ))
    fig.add_trace(go.Scatter(
        x=monthly['year_month'], y=monthly['total_cases'],
        mode='lines', name='Total Cases', line=dict(color='#636EFA')
    ))
    fig.add_trace(go.Scatter(
        x=monthly['year_month'], y=monthly['total_vaccinations'],
        mode='lines', name='Total Vaccinations', line=dict(color='#00CC96')
    ))
    fig.add_trace(go.Scatter(
        x=monthly['year_month'], y=monthly['new_cases'],
        mode='lines', name='New Cases', line=dict(color='#AB63FA')
    ))
    fig.update_layout(
        title=f"{country}: Total Deaths, Cases, Vaccinations, New Cases by Month",
        yaxis_title="Count",
        xaxis_title="Year-Month",
        template="simple_white"
    )
    return fig


## Run the Dash App
Start the Dash app server (for notebook, use JupyterDash and run inline).

In [15]:
if __name__ == '__main__':
    app.run(debug=True, port=8080)

Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/home/docker/Dashboard/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3678: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

